In [ ]:
!pip install sklearn

In [ ]:
!pip install pandas

In [1]:
import tensorflow as tf
from tensorflow import keras #import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPool2D
import numpy as np
from tensorflow.keras import Model

In [13]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,LSTM,TimeDistributed
from keras.layers import Convolution2D, MaxPooling2D,MaxPooling1D,Conv1D
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPool2D
from keras.optimizers import Adam,SGD
from keras.utils import np_utils
from sklearn import metrics
import random
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
import datetime


def reSample(data, samples):
    r = len(data)/samples #re-sampling ratio
    newdata = []
    for i in range(0,samples):
        newdata.append(data[int(i*r)])
    return np.array(newdata)
  
  
train_subjects = ['s07', 's16', 's09', 's13', 's04', 's11', 's15', 's01', 's12', 's10', 's06', 's08']
validation_subjects = ['s02', 's03']
test_subjects = ['s05', 's17']

def get_data(path,sampleSize):
    
    mergedActivities = ['Drinking', 'Eating', 'LyingDown', 'OpeningPillContainer', 
                          'PickingObject', 'Reading', 'SitStill', 'Sitting', 'Sleeping', 
                          'StandUp', 'UseLaptop', 'UsingPhone', 'WakeUp', 'Walking', 
                          'WaterPouring', 'Writing']
    
    specificActivities = ['Calling', 'Clapping', 'Falling', 'Sweeping', 'WashingHand', 'WatchingTV']
    
    enteringExiting = ['Entering', 'Exiting']
    
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []
    X_validation = []
    Y_validation = []
    
    ## Note that 'stft_257_1' contains the STFT features with specification specified in the medium article; 
    ## https://medium.com/@chathuranga.15/sound-event-classification-using-machine-learning-8768092beafc
    
    for file in os.listdir(path + 'stft_257_1/'):
        if int(file.split("__")[1].split("_")[0])!=1:
          a = (np.load(path + "stft_257_1/" + file)).T
          label = file.split('_')[-1].split(".")[0]
          if(label in specificActivities):
              #if(a.shape[0]>100 and a.shape[0]<500):
                if file.split("_")[0] in train_subjects:
#                   X_train.append(reSample(a,sampleSize))
                  X_train.append(np.mean(a,axis=0))
                  Y_train.append(label)
                elif file.split("_")[0] in validation_subjects:
                  X_validation.append(np.mean(a,axis=0))
                  Y_validation.append(label)
                else:
                  X_test.append(np.mean(a,axis=0))
                  Y_test.append(label)
                  #samples[label].append(reSample(a,sampleSize))
          elif(label in enteringExiting):
                label = "enteringExiting"
              #if(a.shape[0]>100 and a.shape[0]<500):
                if file.split("_")[0] in train_subjects:
                  X_train.append(np.mean(a,axis=0))
                  Y_train.append(label)
                elif file.split("_")[0] in validation_subjects:
                  X_validation.append(np.mean(a,axis=0))
                  Y_validation.append(label)
                else:
                  X_test.append(np.mean(a,axis=0))
                  Y_test.append(label)
                  #samples[label].append(reSample(a,sampleSize))
          else:
                label = "other"
              #if(a.shape[0]>100 and a.shape[0]<500):
                if file.split("_")[0] in train_subjects:
                  X_train.append(np.mean(a,axis=0))
                  Y_train.append(label)
                elif file.split("_")[0] in validation_subjects:
                  X_validation.append(np.mean(a,axis=0))
                  Y_validation.append(label)
                else:
                  X_test.append(np.mean(a,axis=0))
                  Y_test.append(label)
                  
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)
    X_validation = np.array(X_validation)
    Y_validation = np.array(Y_validation)
    
    return X_train,Y_train,X_validation,Y_validation,X_test,Y_test
  
def print_M(conf_M):
        s = "activity,"
        for i in range(len(conf_M)):
            s += lb.inverse_transform([i])[0] + ","
        print(s[:-1])
        for i in range(len(conf_M)):
            s = ""
            for j in range(len(conf_M)):
                s += str(conf_M[i][j])
                s += ","
            print(lb.inverse_transform([i])[0],",", s[:-1])
        print()
        
def print_M_P(conf_M):
        s = "activity,"
        for i in range(len(conf_M)):
            s += lb.inverse_transform([i])[0] + ","
        print(s[:-1])
        for i in range(len(conf_M)):
            s = ""
            for j in range(len(conf_M)):
                val = conf_M[i][j]/float(sum(conf_M[i]))
                s += str(round(val,2))
                s += ","
            print(lb.inverse_transform([i])[0],",", s[:-1])
        print()        
        
def showResult():
  predictions = [np.argmax(y) for y in result]
  expected = [np.argmax(y) for y in y_test]

  conf_M = []
  num_labels=y_test[0].shape[0]
  for i in range(num_labels):
      r = []
      for j in range(num_labels):
          r.append(0)
      conf_M.append(r)

  

  n_tests = len(predictions)
  for i in range(n_tests):        
      conf_M[expected[i]][predictions[i]] += 1

  print_M(conf_M)
  print_M_P(conf_M)

In [14]:
featuresPath = "STFT_features/"

a,b,c,d,e,f = get_data(featuresPath,250)

In [15]:
X_train,Y_train,X_validation,Y_validation,X_test,Y_test = a,b,c,d,e,f

n_samples = len(Y_train)
print("No of training samples: " + str(n_samples))
order = np.array(range(n_samples))
np.random.shuffle(order)
X_train = X_train[order]
Y_train = Y_train[order]

lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(Y_train))
y_test = np_utils.to_categorical(lb.fit_transform(Y_test))
y_validation = np_utils.to_categorical(lb.fit_transform(Y_validation))
num_labels = y_train.shape[1]

No of training samples: 880


In [19]:
print(X_train.shape)

(880, 1025)


In [22]:
num_labels = y_train.shape[1]
filter_size = 2

# build model
model = Sequential()

model.add(Dense(256, input_shape=(1025,))) #257
model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
# model.summary()


In [24]:
model.fit(
    X_train, 
    y_train, 
    batch_size=10, 
    epochs=60,
    validation_data=(X_validation,y_validation))

Epoch 1/60


ValueError: in user code:

    /Users/park-yujin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /Users/park-yujin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/park-yujin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/park-yujin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/park-yujin/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pr

In [ ]:
result = model.predict(X_test)

cnt = 0
for i in range(len(Y_test)):
    if(np.amax(result[i])<0.5):
#       pred = 11
      pred = np.argmax(result[i])
    else:
      pred = np.argmax(result[i])
    if np.argmax(y_test[i])==pred:
        cnt+=1

acc = str(round(cnt*100/float(len(Y_test)),2))
print("Accuracy: " + acc + "%")

showResult()

In [ ]:
dir_path = "Models/audio_NN_New"
try:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
except OSError:
    print("Error: Creating Directory: " + dir_path)

In [ ]:
## save model (optional)

path = dir_path+datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
model_json = model.to_json()
with open(path+"_acc_"+acc+".json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(path+"_acc_"+acc+".h5")

In [ ]:
print(path)